In [374]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [375]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [376]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics

In [377]:
chunksize = 10000
df_raw = pd.read_csv("/content/drive/My Drive/Huawei/SearchEngine/train_dataset.csv", chunksize=chunksize,sep='\t')

chunk_list = []  # append each chunk df here 

# Each chunk is in df format
for chunk in df_raw:  
    # perform data filtering 
    #chunk_filter = chunk_preprocessing(chunk)
    
    # Once the data filtering is done, append the chunk to list
    chunk_list.append(chunk)
    break;
    
# concat the list into dataframe 
df_raw = pd.concat(chunk_list)

In [378]:
liste = []
for i in range(365):
  liste.append(str(i))
df_raw.columns = liste

In [379]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [380]:
display_all(df_raw.tail())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364
9995,1,c7d7e06b-71be-11ea-bd9b-2d8f6a4a9b3b,a32f605bb29a4da57365d5560489692d,3,1,0.333,340.01636,0,309,28,70,72,35,1,0,1,16,47,207,255,6,17,1,57,2,2,0,4,-1,-1,9,1,30,7,14,1,3,14,1,0.625000,0.920886,3,1.000000,1.00,3,1.000000,1.00,1355,0.041667,-1.0,-1.0,2,0.666667,0.93,0,0.000000,0.00,0.019608,6,-1.000000,-1.0,49,25,7,16,2,29,0,289,3,108,108,0,0,0,0,108,0,1573705116,14.26,31.27,0.00,0.75,19.60,20.59,0.0,0.0,5.71,12.46,0.0,0.0,0.0,0.0,13.02,0.0,0.0,0.0,0.0,50.0,31.27,0.0,0.75,20.59,0.0,12.46,0.0,17.01,6.75,3.0,3.0,5.56,6.30,11.86,11.86,0.00,0.00,0.00,0.00,2.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.93,0.93,0.00,0.00,1.00,1.00,0.0,0.0,0.0,0.00,8.86,8.86,8.86,1.0,0.0,0.0,1.0,0.0,1.31,0.00,3.72,3.07,0.0,23.749560,200.00000,1.453778,10.135136,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.000000,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0,0,0,0.0,0.0,408,734794,1,0,0,0,0,0,0,0,0,0,5,4,0,1,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9996,0,c7dc4e0a-71be-11ea-bd9b-995d59b2e705,8e7c988174967f85ba1070fad674a824,2,1,0.500,368.82898,1344267441,432,37,99,71,17,1,1,5,111,145,109,237,17,17,1,55,16,0,0,4,-1,-1,31,20,1,54,38,41,31,165,1,0.126761,0.239498,1,0.500000,0.37,1,0.500000,0.37,1766,0.014085,-1.0,-1.0,2,1.000000,1.00,0,0.000000,0.00,0.019608,6,19.251259,-1.0,229,41,7,11,11,32,30,373,2,18,18,2,33,0,0,54,0,229464592,0.00,5.24,0.00,0.40,0.00,5.19,0.0,0.0,0.00,2.25,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,50.0,5.24,0.0,0.40,5.19,0.0,2.25,0.0,5.24,2.25,1.0,1.0,4.02,4.29,8.31,8.31,0.00,0.00,0.00,0.00,2.0,0.0,12.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.00,1.00,0.00,0.00,0.37,0.37,0.0,0.0,0.0,0.00,2.77,2.77,2.77,1.0,0.0,0.0,1.0,0.0,0.70,0.00,0.29,0.29,12.0,6.754300,187.40201,1.793046,11.543345,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.000000,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0.0,0,0,0,0,0.0,0.0,408,1145459,1,0,0,0,0,0,0,0,0,0,5,4,0,1,3,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9997,2,ea798a28-71be-11ea-bd9b-7d092d146c99,b66953312cc0bb422de35d861e6834b3,3,1,0.333,335.78278,1544134604,401,86,41,47,28,8,4,6,4,85,170,255,3,

In [381]:
liste[0] = "sifir"

df_raw.columns = liste
df_raw['sifir'] = df_raw['sifir'] + 1

In [382]:
df_raw["sifir"] = np.log(df_raw.sifir)

In [383]:
df_raw["sifir"] 

0       0.693147
1       0.000000
2       0.693147
3       0.000000
4       0.693147
          ...   
9995    0.693147
9996    0.000000
9997    1.098612
9998    0.000000
9999    0.693147
Name: sifir, Length: 10000, dtype: float64

In [384]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics
import math

In [385]:
m = RandomForestRegressor(n_jobs=-1)
m.fit(df_raw.drop('1', axis=1), df_raw['1'])
#ceviremiyor

ValueError: ignored

In [ ]:
import re
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [ ]:
df_compare = df_raw.copy()

In [ ]:
df_compare.drop("2", axis=1, inplace=True)

In [ ]:
df_raw.items()

In [ ]:
def train_cats(df):
    for n,c in df.items():
        if is_string_dtype(c):
            df[n] = c.astype("category").cat.as_ordered()

In [ ]:
def apply_cats(df, train):
    for n, c in df.items():
        if train[n].dtype == "category":
            df[n] = pd.Categorical(c, categories = train[n].cat.categories, ordered = True)

In [ ]:
df = df_raw.copy()

In [ ]:
df.head()

In [ ]:
train_cats(df)

In [ ]:
train_cats(df_compare)

In [ ]:
df.head()

In [ ]:
import os 

In [ ]:
os.makedirs('tmp', exist_ok=True) # Don't give error if file is already there 
df.to_feather('tmp/bulldozers_1')

In [ ]:
df = pd.read_feather("tmp/bulldozers_1")

In [ ]:
display_all(df.isnull().sum().sort_index()/len(df))

In [ ]:
def fix_missing(df, col, name):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum:
            df[name+"_na"] = pd.isnull(col)
        df[name] = col.fillna(col.median())

In [ ]:
def numericalize(df, col, name):
    if not is_numeric_dtype(col):
        df[name] = col.cat.codes+1

In [ ]:
def proc_df(df, y_fld):
    
    y = df[y_fld].values
    df.drop([y_fld], axis = 1, inplace = True)
    
    for n, c in df.items():
        fix_missing(df, c, n)
        
    for n, c in df.items():
        numericalize(df, c, n)
    
    res = [df, y]
    

    
    return res

In [ ]:
df = pd.read_feather("tmp/bulldozers_1")

In [ ]:
df = df.astype(np.float16) 

In [ ]:
df.head()

In [ ]:
df, y = proc_df(df, '1')
#y cikisini 1 olarak besledim simdilik cunku henuz net degil neyi besleyecegimiz

In [ ]:
df_compare, y_compare = proc_df(df_compare, '1')

In [ ]:
df.columns

In [ ]:
y

In [ ]:
display_all(df.tail().T)

In [ ]:
df = df.dropna()

df


In [ ]:


# Creating validation set
# It will split our data set to have length n train and len(df) - n validation set
def split_train_val(df,n): 
    
    return df[:n].copy(), df[n:].copy()

In [ ]:

n_valid = 12000  # same as Kaggle's test set size
n_train = len(df)-n_valid
X_train, X_valid = split_train_val(df_compare, n_train)
y_train, y_valid = split_train_val(y_compare, n_train)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

In [ ]:
def rmse(x,y): 
    return math.sqrt(((x-y)**2).mean())

In [ ]:
def print_score(m):
    
    print(f"RMSE of train set {rmse(m.predict(X_train), y_train)}")
    print(f"RMSE of validation set {rmse(m.predict(X_valid), y_valid)}")
    print(f"R^2 of train set {m.score(X_train, y_train)}")
    print(f"R^2 of validation set {m.score(X_valid, y_valid)}")

In [ ]:
print(df.columns)

In [ ]:
print(df.dtypes)

In [ ]:
df = df.astype(np.float16) 

In [ ]:
df.dtypes

In [386]:
m = RandomForestRegressor(n_jobs=-1)
%time m.fit(X_train, y_train)
print_score(m)

ValueError: ignored